In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from google.cloud import storage
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Function to get data

def get_stock_data_from_gcp(nrows=10000, local=False, optimize=False, **kwargs):
    """method to get the training data (or a portion of it) from google cloud bucket"""
    # Add Client() here
    client = storage.Client()
    if local:
        path = "data/data_data_10Mill.csv"
    else:
        path = "gs://stock-news-pred-bucket/amazon_stock_price.csv"
    df = pd.read_csv(path)#add nrows after if you want to select a specific number of row
    return df

In [3]:
df = get_stock_data_from_gcp()

In [4]:
X = df[['AMZN']]

In [5]:
# Function to scale data

def scale_stock_data():
    X = df[['AMZN']]
    sc = MinMaxScaler()
    X_sc = sc.fit_transform(X)
    return X_sc

In [6]:
X_sc = scale_stock_data()

In [7]:
# Function to split data

def split_stock_data(train_size):
    index = round(train_size*X.shape[0])
    X_sc_train = X_sc[:index]
    X_sc_test = X_sc[index:]
    return X_sc_train, X_sc_test

In [8]:
X_sc_train, X_sc_test = split_stock_data(0.8)

In [9]:
print(X_sc_train.shape)
print(X_sc_test.shape)

(4442, 1)
(1110, 1)


In [10]:
# Function to reshape train data

def reshape_train_data(obs):
    X_train = []
    y_train = []
    for i in range(obs, X_sc_train.shape[0]):
        X_train.append(X_sc_train[i-obs:i, 0])
        y_train.append(X_sc_train[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train

In [11]:
X_train, y_train = reshape_train_data(30)

In [12]:
print(X_train.shape)
print(y_train.shape)

(4412, 30, 1)
(4412,)


In [13]:
# Function to reshape test data

def reshape_test_data(obs):
    X_test = []
    y_test = []
    for i in range(obs, X_sc_test.shape[0]):
        X_test.append(X_sc_test[i-obs:i, 0])
        y_test.append(X_sc_test[i, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return X_test, y_test

In [14]:
X_test, y_test = reshape_test_data(30)

In [15]:
print(X_test.shape)
print(y_test.shape)

(1080, 30, 1)
(1080,)


In [16]:
# Function to inverse transform

def inverse_transformer(y):
    y = np.c_[y, np.ones(len(y))]
    y = scaler.inverse_transform(y)
    y = y[:,0]
    return y

In [17]:
# Function to build the model

def lstm_model():
    model = Sequential()
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

In [18]:
#model = lstm_model()

In [19]:
# Function to train the model

def train_model():
    model = lstm_model()
    es = EarlyStopping(patience=30, restore_best_weights=True)
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[es], verbose=1)
    return model

In [20]:
# Function to predict and plot results

def predict_and_plot():
    train_model()
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    y_transformed = inverse_transformer(y_test)    
    plt.figure(figsize=(20, 6))
    plt.plot(y_test_transformed, color = 'black', label = 'AMZN Stock Price')
    plt.plot(test_predicted_stock_price, color = 'green', label = 'Predicted AMZN Stock Price')
    plt.title('AMZN Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('AMZN Stock Price')
    plt.legend()
    return plt.show()

In [21]:
train_model()

Epoch 1/100


2021-11-30 16:49:00.506314: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-30 16:49:00.576379: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


111/111 [==============================] - 12s 50ms/step - loss: 0.0063 - val_loss: 3.6851e-05
Epoch 2/100
111/111 [==============================] - 4s 40ms/step - loss: 0.0018 - val_loss: 3.8359e-05
Epoch 3/100
111/111 [==============================] - 5s 42ms/step - loss: 0.0017 - val_loss: 4.8381e-06
Epoch 4/100
111/111 [==============================] - 5s 47ms/step - loss: 0.0013 - val_loss: 5.8744e-05
Epoch 5/100
111/111 [==============================] - 5s 44ms/step - loss: 0.0015 - val_loss: 1.5050e-05
Epoch 6/100
111/111 [==============================] - 4s 39ms/step - loss: 0.0014 - val_loss: 1.3637e-04
Epoch 7/100
111/111 [==============================] - 4s 39ms/step - loss: 0.0014 - val_loss: 3.7408e-05
Epoch 8/100
111/111 [==============================] - 4s 39ms/step - loss: 0.0011 - val_loss: 4.3352e-05
Epoch 9/100
111/111 [==============================] - 4s 40ms/step - loss: 0.0012 - val_loss: 1.0541e-05
Epoch 10/100
111/111 [==============================] - 5

In [26]:
predict_and_plot()

NameError: name 'model' is not defined

In [23]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
y_transformed = inverse_transformer(y_test)    
plt.figure(figsize=(20, 6))
plt.plot(y_test_transformed, color = 'black', label = 'AMZN Stock Price')
plt.plot(test_predicted_stock_price, color = 'green', label = 'Predicted AMZN Stock Price')
plt.title('AMZN Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('AMZN Stock Price')
plt.legend()
plt.show()

NameError: name 'model' is not defined